# Calculating Metrics to Evaluate VS performance

This notebook applies the different metrics reviewed in the previous notebook to the VS scoring results and predictions given by different docking tools.

In [1]:
import pandas as pd
import numpy as np
import glob, os, sys
sys.path.append(r'..')

In [2]:
from modules.plotting_metrics import PlotMetric

## Loading Docking Results
> ### Docking Score & Ligand Efficiency
First, we have to load the docking results, which were directly obtained from the docking tools. Then, we  also load the *Ligand Efficiency Scores*, which were calculated in a previous notebook and correspond to the quotient between *docking score* and the *number of heavy atoms* of each ligand.

Therefore, we load the data:

In [3]:
# CSAR DockingScores
docking_tools_csar = ['AD4_LE', 'AD4_LC', 'VINA', 'VINARDO']
csar_dk_Scores = {dk_tool: pd.read_csv(F'../data/vs_docking_crys_ensemble_{dk_tool}.csv', index_col=0)
                 for dk_tool in docking_tools_csar}
# CSAR Ligand Efficiency
csar_ligEff_Scores = {dk_tool: pd.read_csv(
    F'../data/docking_scores/vs_dk_CRYS_402_CSAR_{dk_tool}_ligand_Efficiency.csv', index_col=0)
                 for dk_tool in docking_tools_csar}

We also need to define the `y_true_csar` variable, which indicates if a given molecule is truly **active** (=1) or **inactive** (=0) accordingly to the experimental report. We also obtain the *pdb_ids* of all conformations.

In [21]:
# TRUE CSAR VALUES
y_true_csar = np.array(csar_dk_Scores['VINA'].ActiveInactive == "Active", dtype=int)
# PDB_IDS
pdb_ids = csar_dk_Scores['VINA'].drop('ActiveInactive', axis = 1).columns


## Evaluation of VS performance

Then we calculate the available metrics to evaluate the VS performance of the following:

### For CSAR:
1. **We have 402 CDK2 protein conformations**

- **We have 4 docking tools/methods**:
    - *AD4 LC*
    - *AD4 LE*
    - *VINA*
    - *VINARDO*
    
- **We have two different _ranking features_**:
    - *Docking Score*: The scores directly obtained from the each of the docking tools.
    - *Ligand Efficiency*: The quotient between *docking score* and the *number of heavy atoms* of each ligand.
    
- **We evaluate four different metrics**:
    - *ROC-AUC*
    - *BEDROC (alpha = 4)*
    - *Precision and Recall AUC*
    - *Normalized Enrichment Factor AUC*


This means that for each of the protein conformations we have calculated 8 different *rankings*, i.e., we evaluated each conformation with 4 docking tools/methods, following by applying two different *ranking features* (directly using the Dk score or by using Ligand Efficiency). And finally each one of these rankings is evaluated by 4 different metrics.

In [22]:
# CSAR metrics results
metrics = {'ROC': 'roc_auc', 'PrcRll': 'pr_auc', 'NEF': 'ref_auc', 'BEDROC': 'bedroc'}
column_to_drop = 'ActiveInactive'
ranking_scores_dfs = {'DkS': csar_dk_Scores, 'DkLEff': csar_ligEff_Scores}

bedroc_alpha = 4

results_dic = {}

for ranking_score in ranking_scores_dfs.keys():
    for metric in metrics.keys():
        for dk_tool in docking_tools_csar:
            rks = ranking_scores_dfs[ranking_score]
            y_preds_dic = rks[dk_tool].drop([column_to_drop], axis = 1).to_dict('list')
            scores = PlotMetric(y_true = y_true_csar, y_pred_dict = y_preds_dic)
            
            # Now we get the name of the column
            col_name = F'{dk_tool}-{ranking_score}-{metric}'
            metric_to_use = metrics[metric]
            if metric_to_use == 'bedroc':
                metric_values = scores.format_metric_results(metric_to_use, 
                                                alpha = bedroc_alpha).iloc[:,0].values
            else:
                metric_values = scores.format_metric_results(metric_to_use).iloc[:,0].values
            results_dic[col_name] = metric_values
    
csar_dk_402confs_vs_metrics = pd.DataFrame(results_dic, index = pdb_ids)

As a result we got  a new DataFrame of 402 rows and 32 columns (4 dk tools *times* 2 ranking features *times* 4 evaluation metrics).

In [25]:
print('VS Performance in CSAR Dataset')
csar_dk_402confs_vs_metrics.head()

VS Performance in CSAR Dataset


,AD4_LE-DkS-ROC,AD4_LC-DkS-ROC,VINA-DkS-ROC,VINARDO-DkS-ROC,AD4_LE-DkS-PrcRll,AD4_LC-DkS-PrcRll,VINA-DkS-PrcRll,VINARDO-DkS-PrcRll,AD4_LE-DkS-NEF,AD4_LC-DkS-NEF,...,VINA-DkLEff-PrcRll,VINARDO-DkLEff-PrcRll,AD4_LE-DkLEff-NEF,AD4_LC-DkLEff-NEF,VINA-DkLEff-NEF,VINARDO-DkLEff-NEF,AD4_LE-DkLEff-BEDROC,AD4_LC-DkLEff-BEDROC,VINA-DkLEff-BEDROC,VINARDO-DkLEff-BEDROC
1aq1,0.637,0.726,0.575,0.730,0.491,0.554,0.273,0.407,0.691,0.763,...,0.582,0.625,0.856,0.890,0.775,0.826,0.748,0.806,0.682,0.723
1b38,0.642,0.694,0.546,0.678,0.402,0.481,0.278,0.352,0.671,0.726,...,0.541,0.636,0.802,0.808,0.763,0.822,0.690,0.692,0.656,0.720
1b39,0.465,0.525,0.581,0.765,0.209,0.249,0.400,0.553,0.483,0.554,...,0.553,0.729,0.700,0.733,0.785,0.873,0.553,0.572,0.676,0.790
1buh,0.573,0.660,0.535,0.708,0.343,0.475,0.287,0.398,0.616,0.702,...,0.488,0.625,0.810,0.849,0.733,0.811,0.702,0.752,0.620,0.699
1ckp,0.570,0.677,0.571,0.767,0.314,0.472,0.387,0.617,0.600,0.712,...,0.574,0.742,0.770,0.804,0.792,0.905,0.657,0.694,0.689,0.822


In [17]:
csar_dk_402confs_vs_metrics.shape

(402, 32)

### For DUD:
Pretty much like with CSAR, although DUD has been only evaluated with Vinardo.
1. **We have 402 CDK2 protein conformations**

- **We have 1 docking tools/methods**
    
- **We have two different _ranking features_**
    
- **We evaluated four different metrics**

In [27]:
# DUD DockingScores
docking_tools_dud = ['VINARDO']
dud_dk_Scores = {dk_tool: pd.read_csv(F'../data/vs_docking_DUD2006_vs_402_crys_vinardo_8x.csv', index_col=0)
                 for dk_tool in docking_tools_dud}
# DUD Ligand Efficiency
dud_ligEff_Scores = {dk_tool: pd.read_csv(
    F'../data/docking_scores/vs_dk_CRYS_402_DUD2006_{dk_tool}_ligand_Efficiency.csv', index_col=0)
                 for dk_tool in docking_tools_dud}

**True values of DUD.**

In [34]:
# TRUE CSAR VALUES
y_true_dud = np.array(dud_dk_Scores['VINARDO'].Actividad == "Active", dtype=int)

In [39]:
# DUD metrics results
docking_tools = docking_tools_dud
metrics = {'ROC': 'roc_auc', 'PrcRll': 'pr_auc', 'NEF': 'ref_auc', 'BEDROC': 'bedroc'}
column_to_drop = 'Actividad'
ranking_scores_dfs = {'DkS': dud_dk_Scores, 'DkLEff': dud_ligEff_Scores}
bedroc_alpha = 20
y_true = y_true_dud

results_dic = {}
for ranking_score in ranking_scores_dfs.keys():
    for metric in metrics.keys():
        for dk_tool in docking_tools:
            rks = ranking_scores_dfs[ranking_score]
            y_preds_dic = rks[dk_tool].drop([column_to_drop], axis = 1).to_dict('list')
            scores = PlotMetric(y_true = y_true, y_pred_dict = y_preds_dic)
            
            # Now we get the name of the column
            col_name = F'{dk_tool}-{ranking_score}-{metric}'
            metric_to_use = metrics[metric]
            if metric_to_use == 'bedroc':
                metric_values = scores.format_metric_results(metric_to_use, 
                                                alpha = bedroc_alpha).iloc[:,0].values
            else:
                metric_values = scores.format_metric_results(metric_to_use).iloc[:,0].values
            results_dic[col_name] = metric_values
    
dud_dk_402confs_vs_metrics = pd.DataFrame(results_dic, index = pdb_ids)

In [40]:
dud_dk_402confs_vs_metrics.head()

,VINARDO-DkS-ROC,VINARDO-DkS-PrcRll,VINARDO-DkS-NEF,VINARDO-DkS-BEDROC,VINARDO-DkLEff-ROC,VINARDO-DkLEff-PrcRll,VINARDO-DkLEff-NEF,VINARDO-DkLEff-BEDROC
1aq1,0.624,0.089,0.623,0.253,0.684,0.148,0.684,0.330
1b38,0.566,0.179,0.572,0.294,0.661,0.205,0.664,0.358
1b39,0.603,0.194,0.608,0.348,0.701,0.191,0.702,0.357
1buh,0.470,0.036,0.470,0.087,0.572,0.049,0.570,0.118
1ckp,0.639,0.132,0.639,0.330,0.732,0.147,0.730,0.348


# Visualization

## CSAR Databases vs 402 CDK2 conformations



In [56]:
df = dud_dk_402confs_vs_metrics.filter(regex='-ROC', axis = 1)
sns.set(style="ticks", color_codes=True)
sns.pairplot(df, markers="+")

In [42]:
import seaborn as sns
import matplotlib.pyplot as plt
fig, axes = plt.subplots()

data = dud_dk_402confs_vs_metrics.filter(regex='ROC', axis = 1)
data = data.melt()
#data['Conf Labels'] = np.tile(df_cdk2_labels.Labels_conf, 6)

fig.set_size_inches(16, 9)
sns.swarmplot(x = "variable", y = "value", data= data, ax = axes, alpha=0.8, 
               palette=  'Set2', s = 6.5)
axes.set_title('AUC: 402 conf, 111 ligs, 3 programas docking')
axes.grid(linestyle='--', linewidth='0.8')
plt.xlabel("Programas de docking")
plt.ylabel("AUC")
plt.ylim(0, 1)
plt.show()


In [7]:
csar_dkScores_confs = csar_dk_Scores['VINA'].drop(['ActiveInactive'], axis = 1).to_dict('list')

In [8]:
csar_dkScores_vina = PlotMetric(y_true = y_true_csar, y_pred_dict = csar_dkScores_confs)
csar_dkScores_vina

In [9]:
csar_dkScores_vina.format_metric_results().iloc[:,0].values

array([0.575, 0.546, 0.581, 0.535, 0.571, 0.607, 0.626, 0.621, 0.548,
       0.577, 0.435, 0.533, 0.467, 0.514, 0.596, 0.533, 0.582, 0.575,
       0.583, 0.426, 0.505, 0.476, 0.573, 0.578, 0.589, 0.507, 0.518,
       0.562, 0.488, 0.55 , 0.535, 0.61 , 0.548, 0.551, 0.436, 0.514,
       0.544, 0.467, 0.522, 0.582, 0.527, 0.519, 0.552, 0.525, 0.568,
       0.63 , 0.643, 0.548, 0.715, 0.482, 0.498, 0.505, 0.522, 0.526,
       0.544, 0.637, 0.54 , 0.643, 0.481, 0.657, 0.591, 0.522, 0.467,
       0.53 , 0.529, 0.581, 0.609, 0.555, 0.473, 0.578, 0.534, 0.45 ,
       0.626, 0.589, 0.61 , 0.608, 0.613, 0.561, 0.566, 0.456, 0.563,
       0.543, 0.606, 0.518, 0.529, 0.541, 0.504, 0.595, 0.526, 0.54 ,
       0.527, 0.577, 0.52 , 0.509, 0.549, 0.561, 0.509, 0.62 , 0.549,
       0.549, 0.558, 0.587, 0.556, 0.631, 0.609, 0.619, 0.587, 0.607,
       0.66 , 0.591, 0.551, 0.519, 0.563, 0.481, 0.57 , 0.463, 0.6  ,
       0.424, 0.527, 0.608, 0.542, 0.6  , 0.612, 0.565, 0.53 , 0.492,
       0.623, 0.544,

In [10]:
for pdb_id in pdb_ids:
    csar_dkScores['VINA'][pdb_id]

NameError: name 'csar_dkScores' is not defined

In [ ]:
pdb_ids = csar_dkScores['VINA'].drop(['ActiveInactive'], axis = 1).columns

#for pdb_id in pdb_ids:

In [ ]:
l = [1, 2, 3]
np.array(np.array(l)) == np.array(l)

In [ ]:
# Generamos los tres casos de CSAR
from truchon_2006_3_ranking_cases import get_three_ranking_cases
# Creamos el arreglo de valores verdaderos:
y_true_csar = np.array([1 if i < 10 else 0 for i in range(100)])
csar_cases_preds = get_three_ranking_cases(y_true_csar, include_optimal = True)
csar_cases = PlotMetric(y_true = y_true_csar, y_pred_dict = csar_cases_preds)
#csar_cases.plot_actives_distribution()

In [ ]:
csar_cases.format_metric_results('ref_auc')

In [ ]:
roc_auc, pr_auc, 

In [15]:
%%html
<link rel="stylesheet" href="https://unpkg.com/treeflex/dist/css/treeflex.css">

<style>
  /* make the nodes round and change their background-color */

  .tf-custom .tf-nc {
    height: 1em; width: 1em;
    background-color: dodgerblue;
    border-color: dodgerblue;
    padding: 0;
    border-radius: 50%;
  }

  /* make the horizontal and vertical connectors thick and change their color */

  .tf-custom .tf-nc:before,
  .tf-custom .tf-nc:after {
    border-left-color: dodgerblue;
    border-left-width: 2px;
  }

  .tf-custom li li:before {
    border-top-color: dodgerblue;
    border-top-width: 2px;
  }
</style>